In [1]:
import sys
import numpy as np
import cumm.tensorview as tv
import spconv.pytorch as spconv

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import pickle
sys.path.append('./origin_data/gpuVersion/')

In [2]:
with open('./origin_data/AxisAlignedTargetAssigner_GPU.pkl', 'rb') as f:
    data = pickle.load(f)
    
points = data['points']
image = data['images']
print(f"points.shape = {points.shape}")
print(f"image.shape = {image.shape}")

points.shape = (19646, 4)
image.shape = (375, 1242, 3)


In [3]:
# plt.figure(figsize=(12,6))
# plt.imshow(image)
# plt.axis('off')

In [4]:
data_path = '/home/jiazx_ug/dataset/S4/O/training/velodyne/005568.bin'
label_path = '/home/jiazx_ug/dataset/S4/O/training/label_2/005568.txt'
points = np.fromfile(data_path, dtype=np.float32).reshape(-1, 4)
print(points.shape)

(69850, 4)


In [5]:
# from visual_utils import plot_pc_data3d,PCD_SCENE, PCD_CAM_VIEW
# lidar_3d_plots = [plot_pc_data3d(x=points[:,0], y=points[:,1], z=points[:,2], colorscale='viridis')]
# layout = dict(template="plotly_dark", scene_camera = PCD_CAM_VIEW, scene = PCD_SCENE, title="POINT CLOUD VISUALIZATION")
# fig = go.Figure(data=lidar_3d_plots, layout=layout)
# fig.show()

# Voxel Generation

In [6]:
vsize_xyz = np.array([0.5, 0.5, 1])                  # voxel size in x,y,z 
coors_range_xyz = np.array([-70.4, -40, -3, 70.4, 40, 1])  # KITTI point cloud range
num_point_features = 4                                 # x,y,z, intensity 
max_num_points_per_voxel = 5               
max_num_voxels = 16000

In [7]:
from spconv.utils import Point2VoxelCPU3d as VoxelGenerator

voxel_generator = VoxelGenerator(
                vsize_xyz=vsize_xyz,
                coors_range_xyz=coors_range_xyz,
                num_point_features=num_point_features,
                max_num_points_per_voxel=max_num_points_per_voxel,
                max_num_voxels=max_num_voxels
            )

# generate voxels from point cloud data
tv_voxels, tv_voxel_indices, tv_num_points = voxel_generator.point_to_voxel(tv.from_numpy(points))

In [8]:
# make copy with numpy(), since numpy_view() will disappear as soon as the generator is deleted
voxels = tv_voxels.numpy()
voxel_indices = tv_voxel_indices.numpy()
num_points = tv_num_points.numpy()

print(f"voxels.shape = {voxels.shape}")
print(f"voxel_indices.shape = {voxel_indices.shape}")
print(f"num_points.shape = {num_points.shape}")

voxels.shape = (7519, 5, 4)
voxel_indices.shape = (7519, 3)
num_points.shape = (7519,)


In [9]:
# converting grid indices to actual 3D coordinates
# indices give bottom left corner, add half voxel size to get voxel centres
voxelCentres = (voxel_indices[:, ::-1] * vsize_xyz) + (coors_range_xyz[0:3]) + (vsize_xyz * 0.5)

# Changing to [x,y,z,dx,dy,dz,yaw] format to find voxel corners
voxelBBoxes = np.column_stack((voxelCentres, np.repeat( np.append(vsize_xyz, 0.0)[None,:], len(voxel_indices), axis=0)))
print(f"voxelBBoxes.shape = {voxelBBoxes.shape}")

from visual_utils import boxes_to_corners_3d
voxelCorners = boxes_to_corners_3d(voxelBBoxes)
print(f"voxelCorners.shape = {voxelCorners.shape}")

voxelBBoxes.shape = (7519, 7)
voxelCorners.shape = (7519, 8, 3)


# Sparse convolution

In [10]:
import torch
import torch.nn as nn
from functools import partial

def meanVFE(voxel_features, voxel_num_points):
    points_mean = voxel_features[:, :, :].sum(dim=1, keepdim=False) # Use sum() instead of _sum()
    normalizer = torch.clamp_min(voxel_num_points.view(-1, 1), min=1.0).type_as(voxel_features)
    points_mean = points_mean / normalizer # sum / count for each voxel
    return points_mean.contiguous()

class voxelbackbone8x(nn.Module):
    def __init__(self, input_channels, grid_size):
        super().__init__()
        norm_fn = partial(nn.BatchNorm1d, eps=1e-3, momentum=0.01)
        self.sparse_shape = grid_size[::-1] + [1, 0, 0]

        self.conv_input = spconv.SparseSequential(
            spconv.SubMConv3d(input_channels, 16, 3, padding=1, bias=False, indice_key="subm0").to(0),
            norm_fn(16).to(0),
            nn.ReLU()
        )

    def forward(self, voxel_features, voxel_coords):
        sp_tensor = spconv.SparseConvTensor(
            features = voxel_features,
            indices = voxel_coords,
            spatial_shape = self.sparse_shape,
            batch_size = 1
        )

        x = self.conv_input(sp_tensor)
        return x

torch_voxels = torch.from_numpy(voxels)
torch_num_points = torch.from_numpy(num_points)
torch_indices = torch.from_numpy(voxel_indices)

mean_vfe = meanVFE(torch_voxels, torch_num_points)
coordinates = torch.cat([torch.zeros_like(torch_indices[:, :1]), torch_indices], dim=1)
for i in range(len(coordinates)):
    coordinates[i, 0] = i

grid_size = (coors_range_xyz[3:6] - coors_range_xyz[0:3]) / np.array(vsize_xyz)
grid_size = np.round(grid_size).astype(np.int64)
# print(grid_size[::] + [0, 0, 1])

# load mean_vfe, coordinates to gpu


backbone = voxelbackbone8x(input_channels=4, grid_size=grid_size)
print(backbone)
torch_voxel_coords = torch.from_numpy(voxel_indices).int()
torch_voxel_features = backbone(mean_vfe.to(0), coordinates.to(0))


voxelbackbone8x(
  (conv_input): SparseSequential(
    (0): SubMConv3d(4, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
    (1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (2): ReLU()
  )
)


In [11]:
# # print backbones parameters
# for name, param in backbone.named_parameters():
#     print(name, param.size())
#     print(param)

In [12]:
indices = torch_voxel_features.indices.int().cpu().numpy()
indices_3d = np.zeros(grid_size, dtype=np.int32)
# /hqlab/public_dataset/cadc/dataset/cadcd
indices_3d[indices[:,3],indices[:,2],indices[:,1]] = indices[:,0]

print(indices_3d.shape)

(282, 160, 4)


In [13]:
with open(label_path, 'r') as f:
    labels = f.readlines()

corners_3Ds = []

for line in labels:
    line = line.split()
    lab, x, y, z, w, l, h, rot = line[0], line[11], line[12], line[13], line[9], line[10], line[8], line[14]
    h, w, l, x, y, z, rot = map(float, [h, w, l, x, y, z, rot])
    
    if lab != 'DontCare':
        x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2]
        y_corners = [0, 0, 0, 0, -h, -h, -h, -h]
        z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2]
        corners_3d = np.vstack([x_corners, y_corners, z_corners])  # (3, 8)

        # transform the 3d bbox from object coordiante to camera_0 coordinate
        R = np.array([[np.cos(rot), 0, np.sin(rot)],
                    [0, 1, 0],
                    [-np.sin(rot), 0, np.cos(rot)]])
   
        corners_3d = np.dot(R, corners_3d).T + np.array([x, y, z])

        # transform the 3d bbox from camera_0 coordinate to velodyne coordinate
        corners_3d = corners_3d[:, [2, 0, 1]] * np.array([[1, -1, -1]])
        corners_3Ds.append(corners_3d)
        

In [14]:
# change corner3d into voxel_indices coordinate, vsize_xyz = np.array([0.5, 0.5, 1]) ,coors_range_xyz = np.array([-70.4, -40, -3, 70.4, 40, 1]) 
corners_voxel = np.array(corners_3Ds)
corners_voxel = (corners_voxel - coors_range_xyz[0:3] - 0.5 * vsize_xyz) / vsize_xyz
# corners_voxel = (corners_voxel - coors_range_xyz[0:3]) / vsize_xyz - 0.5
corners_voxel = corners_voxel.astype(np.int32)
print(corners_voxel.shape)
output_txt = './output_temp.txt'
with open(output_txt, 'w') as f:
    for corner in corners_voxel:
        for c in corner:
            f.write(f'{c[0]} {c[1]} {c[2]}\n')
        f.write('\n')


(5, 8, 3)


In [15]:
class isPointInQuadrangle(object):
    def __int__(self):
        self.__isInQuadrangleFlag = False

    def cross_product(self, xp, yp, x1, y1, x2, y2):
        return (x2 - x1) * (yp - y1)-(y2 - y1) * (xp - x1)

    def compute_para(self, xp, yp, xa, ya, xb, yb, xc, yc, xd, yd):
        cross_product_ab = isPointInQuadrangle().cross_product(xp, yp, xa, ya, xb, yb)
        cross_product_bc = isPointInQuadrangle().cross_product(xp, yp, xb, yb, xc, yc)
        cross_product_cd = isPointInQuadrangle().cross_product(xp, yp, xc, yc, xd, yd)
        cross_product_da = isPointInQuadrangle().cross_product(xp, yp, xd, yd, xa, ya)
        return cross_product_ab,cross_product_bc,cross_product_cd,cross_product_da

    def is_in_rect(self, aa, bb, cc, dd):
        if (aa > 0 and bb > 0 and cc > 0 and dd > 0) or (aa < 0 and bb < 0 and cc < 0 and dd < 0):
            self.__isInQuadrangleFlag= True
        else:
            self.__isInQuadrangleFlag = False

        return self.__isInQuadrangleFlag
    
feature_keys = []
checker = isPointInQuadrangle()
# for corner in corners_voxel:
#     max_x,min_x,max_y,min_y = np.max(corner[:,0]),np.min(corner[:,0]),np.max(corner[:,1]),np.min(corner[:,1])
#     x = np.arange(int(min_x),int(max_x)+1)
#     y = np.arange(int(min_y),int(max_y)+1)
#     for i in x:
#         for j in y:
#             feature_keys.append((i,j,corner[0][2]))
# print(len(feature_keys))

for corner in corners_voxel:
    vector01 = corner[1] - corner[0]
    vector03 = corner[3] - corner[0]
    square_area = np.linalg.norm(np.cross(vector01, vector03))
    max_x,min_x,max_y,min_y = np.max(corner[:,0]),np.min(corner[:,0]),np.max(corner[:,1]),np.min(corner[:,1])
    x = np.arange(int(min_x),int(max_x)+1)
    y = np.arange(int(min_y),int(max_y)+1)
    for i in x:
        for j in y:
            a,b,c,d = checker.compute_para(i,j,corner[0][0],corner[0][1],corner[1][0],corner[1][1],corner[2][0],corner[2][1],corner[3][0],corner[3][1])
            if checker.is_in_rect(a,b,c,d):
                for k in range(corner[0][2],corner[4][2]+1):
                    feature_keys.append((i,j,k))

print(len(feature_keys))

204


In [16]:
features = []
conved_features = torch_voxel_features.features.cpu().detach().numpy()
for key in feature_keys:
    if key[0] < 0 or key[0] >= grid_size[0] or key[1] < 0 or key[1] >= grid_size[1] or key[2] < 0 or key[2] >= grid_size[2]:
        continue
    indice = indices_3d[key[0],key[1],key[2]]
    features.append(conved_features[indice])
print(len(features))    

66


In [17]:
# import random
# from visual_utils import plot_bboxes_3d
# from visual_utils import plot_pc_data3d,PCD_SCENE, PCD_CAM_VIEW
# hexadecimal_alphabets = '0123456789ABCDEF'
# color = ["#" + ''.join([random.choice(hexadecimal_alphabets) for j in range(6)]) for i in range(voxel_indices.shape[0])]

# voxel_generator_plots = [plot_pc_data3d(x=points[:,0], y=points[:,1], z=points[:,2], opacity=0.1, colorscale='viridis')]
# voxel_generator_plots.extend(plot_bboxes_3d(voxelCorners, color))

# for vox_idx in range(voxel_indices.shape[0]):
#     voxel_points = voxels[vox_idx, 0 : num_points[vox_idx], 0:3]
#     print(voxel_points)
#     voxel_generator_plots.append(plot_pc_data3d(x=voxel_points[:,0], y=voxel_points[:,1], z=voxel_points[:,2], \
#                                                 apply_color_gradient=False, color=color[vox_idx]))
    
# for voxel_points in corners_3Ds:
#     voxel_generator_plots.append(plot_pc_data3d(x=voxel_points[:,0], y=voxel_points[:,1], z=voxel_points[:,2], \
#                                                 apply_color_gradient=False, color='red'))

# layout = dict(scene=PCD_SCENE, scene_camera = PCD_CAM_VIEW, title="VOXEL GENERATOR PLOTS", template="plotly_dark")
# fig = go.Figure(data=voxel_generator_plots, layout=layout)
# fig.show()  

In [18]:
# plt.figure(figsize=(8,8))
# plt.scatter(points[:,1], points[:,0], alpha=0.1, s=2)

# for vox_idx in range(len(voxel_indices)):
#     voxel_points = voxels[vox_idx, 0 : num_points[vox_idx], 0:3]
#     plt.scatter(voxel_points[:,1], voxel_points[:,0], s=2, label=f"Voxel_{vox_idx}")
#     voxelBoxCornersBEV = voxelCorners[vox_idx, [0,1,2,3,0], 0:2]
#     plt.plot(voxelBoxCornersBEV[:,1], voxelBoxCornersBEV[:,0], label=f"Voxel_BBox{vox_idx}")
# plt.show()